In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Configurações de conexão
user = 'luiz'
password = 't5NajFQr9COpyP'
host = 'db.nqriypvbuemqemsuhisk.supabase.co'
port = 5432
database = 'postgres'

# String de conexão
conn_str = f'postgresql://{user}:{password}@{host}:{port}/{database}'
engine = create_engine(conn_str)

# Substitua 'nome_da_tabela' pela tabela que você deseja consultar
query1 = 'SELECT * FROM review'
query2 = 'SELECT * FROM aspect'

# Carregar os dados em um DataFrame pandas
df = pd.read_sql(query1, engine)
df.dropna(subset=["review_rating", "review_text"], inplace=True)
df = df[df["review_text"].str.strip() != ""]
df["review_posted_at"] = pd.to_datetime(df["review_posted_at"], errors='coerce')

df2 = pd.read_sql(query2, engine)
df2.dropna(subset=["name"], inplace=True)
df2 = df2[df2["name"].str.strip() != ""]

df_final = pd.merge(df, df2, right_on="review_id", left_on="id", how="inner")

# Exibir o DataFrame
df


,id,source_name,review_title,review_text,review_content,review_rating,review_posted_at,review_hotel_reply,reservation_check_in,reservation_nights,reservation_type,user_name,user_country,user_language,hotel_id,raw_review_id,created_at,updated_at
0,f8334ba6-5eed-4f59-a9b3-6651bced95ca,expedia,,"Quartos bons, ar condicionado bem silencioso, ...","Quartos bons, ar condicionado bem silencioso, ...",8.0,2025-01-19 00:00:00+00:00,"Olá Isabela, agradecemos muito pelo seu feedba...",None,NaN,Família,Isabela,Brasil,pt,viale_cataratas,678d6d0420bfcf324b10fdc8,2025-07-23 18:53:20.655265+00:00,2025-07-23 18:53:20.655265+00:00
1,3d90c397-e7ac-4a29-b715-f38de1f7c67a,booking,,"<+> Все очень прекрасно, дружелюбный персонал ...","O que gostei: Все очень прекрасно, дружелюбный...",10.0,2025-07-23 09:17:12+00:00,,2025-05-24,1.0,Família,Ксения,Rússia,ru,carima,f6c4a3fdcf087e76,2025-07-23 17:24:12.008727+00:00,2025-07-23 17:24:12.008727+00:00
2,5baaabe0-e587-49e0-8527-0f73f0e029a0,expedia,Jan/25 Foz do Iguaçu.,"Hotel com ótima estrutura, bom café da manhã, ...",Jan/25 Foz do Iguaçu. Hotel com ótima estrutur...,10.0,2025-01-17 00:00:00+00:00,"Olá Douglas, agradecemos imensamente pelo seu ...",None,NaN,Família,DOUGLAS,Brasil,pt,viale_cataratas,678abbeaeaeee70588d3029b,2025-07-23 18:53:20.655265+00:00,2025-07-23 18:53:20.655265+00:00
3,a6f9a878-cdef-402d-bfc7-ed0724a09bba,google,,"Muy lindo el hotel, el servicio excelente, est...","Muy lindo el hotel, el servicio excelente, est...",10.0,2025-04-21 18:35:58+00:00,"Estimada Laura Lipkindas, agradecemos sinceram...",None,NaN,Família,Laura Lipkindas,None,es,nadai,ChZDSUhNMG9nS0VJQ0FnTURvbU9tV2VnEAE,2025-07-23 18:01:06.244606+00:00,2025-07-23 18:01:06.244606+00:00
4,4f29a994-904c-42cf-96a3-1374f9a80007,expedia,,Muito bom atendimento e o tamanho do quarto.,Muito bom atendimento e o tamanho do quarto.,10.0,2025-06-03 00:00:00+00:00,,None,NaN,None,RAFAEL F,Brasil,pt,bourbon,683f8399de414719955f846e,2025-07-23 18:53:31.435089+00:00,2025-07-23 18:53:31.435089+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22788,311bbaa1-f0e2-46d1-b7ba-1ec77b513f7a,google,,"Maravilhosa, lugar lindo, super bem atendida p...","Maravilhosa, lugar lindo, super bem atendida p...",10.0,2024-07-08 00:50:16+00:00,,None,NaN,Família,Iete Costato Pedro,None,pt,taroba,ChdDSUhNMG9nS0VJQ0FnSUNyb2R5N2xnRRAB,2025-07-23 18:01:44.767921+00:00,2025-07-23 18:01:44.767921+00:00
22789,9846dcd8-62f7-40a6-9176-0cdd4bce4f4e,google,,Show de bola,Show de bola,10.0,2024-07-05 21:24:55+00:00,,None,NaN,Amigos,Joel Raimundo do Nascimento,None,pt,taroba,ChZDSUhNMG9nS0VJQ0FnSUNyeXJMMlFBEAE,2025-07-23 18:01:44.767921+00:00,2025-07-23 18:01:44.767921+00:00
22791,3c930a10-0783-4758-8e17-e5ba73bb1486,google,,Gostei muito da estadia do hotel serviço e ate...,Gostei muito da estadia do hotel serviço e ate...,10.0,2024-01-09 01:40:17+00:00,"Olá, Gisele! Muito obrigada por sua avaliação ...",None,NaN,None,Gisele Cristina,None,pt,taroba,ChdDSUhNMG9nS0VJQ0FnSUQxdGVMbmxBRRAB,2025-07-23 18:01:46.036271+00:00,2025-07-23 18:01:46.036271+00:00
22792,e6ec14b1-4df4-4288-967e-5a1f6dbb5995,google,,"Hotel maravilhoso , ótima recepção\nA camareir...","Hotel maravilhoso , ótima recepção\nA camareir...",10.0,2024-01-07 18:43:27+00:00,"Olá, Fernanda! Esperamos que sua viagem de ret...",None,NaN,Casal,Fernanda Elisa,None,pt,taroba,ChZDSUhNMG9nS0VJQ0FnSUQxb2JlQ0J3EAE,2025-07-23 18:01:12.640659+00:00,2025-07-23 18:01:12.640659+00:00


In [2]:
df.to_csv('review_data.csv', index=False)

In [3]:
df2.to_csv('aspect_data.csv', index=False)

In [4]:
df_for_text_sentiment = df_final[["description", "sentiment", "category", "hotel_id", "user_language", "review_posted_at"]].copy()

for hotel in df_for_text_sentiment["hotel_id"].unique():
    df_hotel = df_for_text_sentiment[df_for_text_sentiment["hotel_id"] == hotel]
    print(df_hotel["sentiment"].unique())

[ 1 -1  0]
[ 1 -1  0]
[ 1 -1  0]
[ 1 -1  0]
[ 1 -1  0]
[-1  1  0]
[-1  1  0]
[ 1 -1  0]


In [5]:
df_for_text_sentiment = df_for_text_sentiment[df_for_text_sentiment["sentiment"] != 0].copy()
df_for_text_sentiment.loc[:, "sentiment"] = df_for_text_sentiment["sentiment"].replace(-1, 0)
df_for_text_sentiment["sentiment"].value_counts()

sentiment
1    27449
0     9435
Name: count, dtype: int64

In [6]:
df_for_text_sentiment = df_for_text_sentiment[df_for_text_sentiment["user_language"] == "pt"]
df_for_text_sentiment

,description,sentiment,category,hotel_id,user_language,review_posted_at
0,ar condicionado bem silencioso,1,Quarto,viale_cataratas,pt,2025-01-19 00:00:00+00:00
1,camas boas,1,Quarto,viale_cataratas,pt,2025-01-19 00:00:00+00:00
2,"Copa baby bem abandonada, estava suja, não tin...",0,Áreas comuns,viale_cataratas,pt,2025-01-19 00:00:00+00:00
5,Hotel com ótima estrutura,1,Experiência,viale_cataratas,pt,2025-01-17 00:00:00+00:00
6,camas confortáveis,1,Quarto,viale_cataratas,pt,2025-01-17 00:00:00+00:00
...,...,...,...,...,...,...
37447,super bem atendida por todos ❤️,1,Atendimento da equipe,taroba,pt,2024-07-08 00:50:16+00:00
37448,"Maravilhosa, lugar lindo",1,Experiência,taroba,pt,2024-07-08 00:50:16+00:00
37449,limpeza,1,Quarto,taroba,pt,2024-01-09 01:40:17+00:00
37450,"A camareira Lourdes com um trabalho impecável,...",1,Atendimento da equipe,taroba,pt,2024-01-07 18:43:27+00:00


In [7]:
df_for_text_sentiment["category"].value_counts()

category
Atendimento da equipe    4363
Quarto                   3525
Áreas comuns             2136
Café da manhã            1734
Restaurante/Bar          1452
Experiência              1376
Localização              1021
Banheiro                  582
Segurança                  75
Estacionamento              1
Name: count, dtype: int64

In [8]:
df_for_text_sentiment.drop(["user_language"], axis=1, inplace=True)

In [9]:
df_final = pd.DataFrame()

for hotel in df_for_text_sentiment["hotel_id"].unique():
    df_hotel = df_for_text_sentiment[df_for_text_sentiment["hotel_id"] == hotel]
    df_hotel.reset_index(drop=True, inplace=True)

    categories_to_keep = df_hotel["category"].value_counts()[lambda x: x >= 100].index
    df_hotel = df_hotel[df_hotel["category"].isin(categories_to_keep)].copy()

    df_final = pd.concat([df_final, df_hotel], ignore_index=True)

df_final["category"].value_counts()

category
Atendimento da equipe    4363
Quarto                   3525
Áreas comuns             2041
Café da manhã            1637
Experiência              1161
Restaurante/Bar          1135
Localização               785
Banheiro                  155
Name: count, dtype: int64

In [10]:
import os

# Save individual CSVs for each hotel with reset index
for h in df_final["hotel_id"].unique():
    folder_path = "datasets/"
    os.makedirs(folder_path, exist_ok=True)
    
    # Filter reviews for the current hotel and reset the index
    df_h = df_final[df_final["hotel_id"] == h].reset_index(drop=True)
    
    # Save CSV file in the corresponding folder
    csv_filename = f"{h}.csv"
    csv_path = os.path.join(folder_path, csv_filename)
    df_h.to_csv(csv_path, index=False)